In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer, DataCollatorWithPadding
# Criando o pipeline de classificação
from transformers import pipeline

In [1]:
model_checkpoint = "neuralmind/bert-base-portuguese-cased"  # ou outro modelo de sua preferência
num_labels = 2  # ajuste conforme o número de classes do seu problema

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

c:\Users\alber\anaconda3\envs\env_gen_ai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",               # diretório para salvar os checkpoints e resultados
    evaluation_strategy="epoch",          # avaliação a cada época
    learning_rate=2e-5,                   # taxa de aprendizado
    per_device_train_batch_size=16,       # batch size para treinamento
    per_device_eval_batch_size=16,        # batch size para avaliação
    num_train_epochs=3,                   # número de épocas de treinamento
    weight_decay=0.01,                    # decaimento de peso
    logging_dir='./logs',                 # diretório para logs
)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
# Iniciando o treinamento
trainer.train()

In [ ]:
# Avaliando o modelo
eval_results = trainer.evaluate()
print("Resultados da avaliação:", eval_results)



classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
# Exemplo de inferência:
resultado = classifier("Este é um exemplo de texto para classificação.")
print("Resultado da classificação:", resultado)
